# Plotly

#### Loading Libraries

In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.reset_option('max_rows')
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

#### Loading Data from dfd.csv (EDA Output)

In [2]:
dfd = pd.read_csv('dfd.csv')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (20,50,60,119,130,131,132,135,136,137,140,146,147,148) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
dfd.head()

,issue_date,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,issue_year,issue_date.1
0,2015-12-01,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,...,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2015,2015-12-01
1,2015-12-01,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,...,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2015,2015-12-01
2,2015-12-01,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,...,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2015,2015-12-01
3,2015-12-01,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,...,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2015,2015-12-01
4,2015-12-01,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,...,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2015,2015-12-01


#### Plotly libraries

In [4]:
import itertools
from sklearn import preprocessing
# import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import plotly
# connected=True means it will download the latest version of plotly javascript library.
plotly.offline.init_notebook_mode(connected=True)

from plotly.graph_objs import Figure, Histogram, Layout

---

In [5]:
def default_flag_fn(loan_status):
    if any([loan_status == 'Default', \
            loan_status == 'Charged Off', \
            loan_status == 'Does not meet the credit policy. Status:Charged Off']):
        return 1
    else:
        return 0

In [6]:
def fully_paid_and_default_flag_fn(loan_status):
    if any([loan_status == 'Default', \
            loan_status == 'Charged Off', \
            loan_status == 'Does not meet the credit policy. Status:Charged Off', \
            loan_status == 'Fully Paid', \
            loan_status == 'Does not meet the credit policy. Status:Fully Paid']):
        return 1
    else:
        return 0

In [7]:
dfd['default_flag'] = dfd['loan_status'].apply(default_flag_fn)

In [8]:
dfd['fully_paid_and_default_flag'] = dfd['loan_status'].apply(fully_paid_and_default_flag_fn)

In [9]:
dfd[dfd['loan_status'] == 'Fully Paid'][['loan_status','default_flag','fully_paid_and_default_flag']].tail()
# dfd[['loan_status','default_flag','fully_paid_and_default_flag']].tail()

,loan_status,default_flag,fully_paid_and_default_flag
2260653,Fully Paid,0,1
2260655,Fully Paid,0,1
2260657,Fully Paid,0,1
2260659,Fully Paid,0,1
2260661,Fully Paid,0,1


In [10]:
# Default Rate by State:

# total numer of charged off / (total number of charged off + fully paid)

def_rate = np.round((dfd.groupby(['addr_state'])['default_flag'].sum() /\
                     dfd.groupby(['addr_state'])['fully_paid_and_default_flag'].sum())\
                    *100,2)

In [11]:
result_df = pd.DataFrame(def_rate, columns =['default_rate'])

In [12]:
result_df.head()    

,default_rate
addr_state,
AK,19.66
AL,23.63
AR,24.11
AZ,19.66
CA,19.63


In [13]:
result_df.reset_index(inplace=True)

In [14]:
for col in result_df.columns:
    result_df[col] = result_df[col].astype(str)

    scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

result_df['text'] = result_df['addr_state'] + '<br>' # +\
#     '<br>'+'Weighted Rate: '+df_plot['Weighted_Rate']+'<br>'+\
#     'Inquiries Last 12m: '+df_plot['12m_Inquiries']+'<br>'+\
#     'Inquiries Last 6m: '+df_plot['6m_Inquiries']

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = True,
        locations = result_df['addr_state'],
        z = result_df['default_rate'], #.astype(int),
        locationmode = 'USA-states',
        text = result_df['text'], 
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "%")
        ) ]

layout = dict(
        title = 'Lending Club Portfolio<br>Default Rate By State',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
iplot( fig, filename='d3-cloropleth-map' )

---